In [1]:
%cd ../../..


/run/media/nazif/2F946E411BA61D49/mirscribe


In [2]:
import pandas as pd
from ast import literal_eval
import json
from scripts.ensembl import *
g37 = import_pyensembl(37)


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /run/media/nazif/2F946E411BA61D49/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [3]:
def get_first_element(x):
    try:
        lst = literal_eval(x)
        return lst[0] if len(lst) > 0 else None
    except (ValueError, SyntaxError):
        return None

In [4]:
df = pd.read_csv("results/from_truba/vcf1_results.csv")

df.head()

,id,mirna_accession,ENST,gain,loss
0,PD3851a__10_103447203_C_A,MIMAT0000069,"['ENST00000331272', 'ENST00000457105']",1,0
1,PD3851a__10_103447203_C_A,MIMAT0000257,"['ENST00000331272', 'ENST00000457105']",1,0
2,PD3851a__10_103447203_C_A,MIMAT0000264,"['ENST00000331272', 'ENST00000457105']",1,0
3,PD3851a__10_103447203_C_A,MIMAT0000275,"['ENST00000331272', 'ENST00000457105']",1,0
4,PD3851a__10_103447203_C_A,MIMAT0000429,"['ENST00000331272', 'ENST00000457105']",1,0


In [5]:
df = pd.read_csv("results/from_truba/vcf1_results.csv")

# preprocessing df

# get chr, pos from id
df[["chr", "pos"]] = df["id"].str.split("_", expand=True).iloc[:, 2:4]

# get first element of ENST
df["ENST_single"] = df["ENST"].apply(get_first_element)

df = df.drop(columns=["ENST"])
df.rename(columns={"ENST_single": "ENST"}, inplace=True)

# getting gene names of every ENST
df["gene_name"] = df["ENST"].apply(lambda x: g37.gene_name_of_transcript_id(x) if x is not None else None)


In [6]:
df[["id", "mirna_accession", "ENST", "gene_name", "gain", "loss"]].head(10)

,id,mirna_accession,ENST,gene_name,gain,loss
0,PD3851a__10_103447203_C_A,MIMAT0000069,ENST00000331272,FBXW4,1,0
1,PD3851a__10_103447203_C_A,MIMAT0000257,ENST00000331272,FBXW4,1,0
2,PD3851a__10_103447203_C_A,MIMAT0000264,ENST00000331272,FBXW4,1,0
3,PD3851a__10_103447203_C_A,MIMAT0000275,ENST00000331272,FBXW4,1,0
4,PD3851a__10_103447203_C_A,MIMAT0000429,ENST00000331272,FBXW4,1,0
5,PD3851a__10_103447203_C_A,MIMAT0000688,ENST00000331272,FBXW4,1,0
6,PD3851a__10_103447203_C_A,MIMAT0000725,ENST00000331272,FBXW4,1,0
7,PD3851a__10_103447203_C_A,MIMAT0000765,ENST00000331272,FBXW4,1,0
8,PD3851a__10_103447203_C_A,MIMAT0001545,ENST00000331272,FBXW4,1,0
9,PD3851a__10_103447203_C_A,MIMAT0001638,ENST00000331272,FBXW4,1,0


In [11]:
# Group mutations by "ENST" (mRNA identifier) and calculate the total gains, losses, and total mutations
mRNA_trends = df.groupby("ENST").agg(
    total_gains=pd.NamedAgg(column="gain", aggfunc="sum"),
    total_losses=pd.NamedAgg(column="loss", aggfunc="sum"),
    total_modifications=pd.NamedAgg(column="id", aggfunc="count")
).reset_index().sort_values("total_modifications", ascending=False)

# enst:gene_name dict
gene_names = dict(zip(df["ENST"], df["gene_name"]))

mRNA_trends["gene_names"] = mRNA_trends["ENST"].apply(
    lambda x: gene_names.get(x, None)
)

mRNA_trends["biotype"] = mRNA_trends["gene_names"].apply(lambda x: g37.genes_by_name(x)[0].biotype if x is not None else None)


mRNA_trends[mRNA_trends.biotype == "protein_coding"].head(10)


,ENST,total_gains,total_losses,total_modifications,gene_names,biotype
556,ENST00000422070,531,444,975,RBFOX1,protein_coding
492,ENST00000389413,518,353,871,PTPRN2,protein_coding
166,ENST00000288447,342,500,842,DMD,protein_coding
634,ENST00000466387,347,455,802,CTNNA2,protein_coding
0,ENST00000066544,283,512,795,CDC27,protein_coding
311,ENST00000331898,414,330,744,OPCML,protein_coding
461,ENST00000377067,330,409,739,GPC5,protein_coding
147,ENST00000281156,409,328,737,KHDRBS2,protein_coding
622,ENST00000458659,146,563,709,TSC22D1,protein_coding
146,ENST00000281127,386,309,695,NRXN3,protein_coding


In [12]:
mRNA_trends[mRNA_trends.biotype == "protein_coding"].head(10).gene_names


556     RBFOX1
492     PTPRN2
166        DMD
634     CTNNA2
0        CDC27
311      OPCML
461       GPC5
147    KHDRBS2
622    TSC22D1
146      NRXN3
Name: gene_names, dtype: object

In [13]:
mutation_trends = (df
                   .groupby("id")
                   .agg(
                       total_gains=pd.NamedAgg(column="gain", aggfunc="sum"),
                       total_losses=pd.NamedAgg(column="loss", aggfunc="sum"),
                       total_modifications=pd.NamedAgg(
                           column="id", aggfunc="count")
                   )
                   .reset_index()
                   .sort_values("total_modifications", ascending=False))

mutation_trends["relative_impact"] = mutation_trends["total_gains"] - \
    mutation_trends["total_losses"]

gene_names_with_id = dict(zip(df["id"], df["gene_name"]))

mutation_trends["gene_names"] = mutation_trends["id"].apply(
    lambda x: gene_names_with_id.get(x, None)
)

mutation_trends["biotype"] = mutation_trends["gene_names"].apply(
    lambda x: g37.genes_by_name(x)[0].biotype if x is not None else None)


mutation_trends[mutation_trends.biotype == "protein_coding"].head(10)

,id,total_gains,total_losses,total_modifications,relative_impact,gene_names,biotype
142,PD3851a__11_79127100_G_C,151,438,589,-287,TENM4,protein_coding
28,PD3851a__10_18262234_T_G,402,138,540,264,SLC39A12,protein_coding
365,PD3851a__14_79918604_C_A,303,211,514,92,NRXN3,protein_coding
1767,PD3851a__X_83118456_G_T,204,296,500,-92,CYLC1,protein_coding
875,PD3851a__2_182541934_G_T,145,339,484,-194,NEUROD1,protein_coding
689,PD3851a__1_236904096_G_T,140,334,474,-194,ACTN2,protein_coding
508,PD3851a__17_40755034_G_C,205,267,472,-62,FAM134C,protein_coding
949,PD3851a__2_80416948_G_T,161,309,470,-148,CTNNA2,protein_coding
1294,PD3851a__5_60428475_G_C,179,284,463,-105,NDUFAF2,protein_coding
721,PD3851a__1_66697391_G_A,49,414,463,-365,PDE4B,protein_coding


In [14]:
mutation_trends[mutation_trends.biotype == "protein_coding"].head(10).gene_names


142        TENM4
28      SLC39A12
365        NRXN3
1767       CYLC1
875      NEUROD1
689        ACTN2
508      FAM134C
949       CTNNA2
1294     NDUFAF2
721        PDE4B
Name: gene_names, dtype: object

# miRNA trends


In [17]:
mirna_trends = df.groupby("mirna_accession").agg(
    total_gains=pd.NamedAgg(column="gain", aggfunc="sum"),
    total_losses=pd.NamedAgg(column="loss", aggfunc="sum"),
    total_modifications=pd.NamedAgg(column="id", aggfunc="count")
).reset_index().sort_values("total_modifications", ascending=False)

mirna_trends.head(10)

,mirna_accession,total_gains,total_losses,total_modifications
1876,MIMAT0022730,143,144,287
1207,MIMAT0018447,143,144,287
2068,MIMAT0026736,136,150,286
1411,MIMAT0019693,155,127,282
1845,MIMAT0022692,157,123,280
2497,MIMAT0030428,147,131,278
2044,MIMAT0026618,123,154,277
1727,MIMAT0021036,152,124,276
851,MIMAT0009201,127,149,276
132,MIMAT0000460,142,133,275
